In [1]:
# Load station locations fro station metadata database and sanity check them against seismic events.
%matplotlib inline

In [2]:
import os
import sys
sys.path.append(os.path.realpath('..'))
from collections import defaultdict

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,10)

In [4]:
import obspy
# This is a bit dodgy since it assumes sperical earth...
from obspy.geodetics.base import locations2degrees

In [19]:
from tqdm.auto import tqdm

In [20]:
import random
random.seed(20190118)

In [5]:
from dataio.catalogcsv import CatalogCSV

In [6]:
event_src_folder = r'/g/data/ha3/am7399/temp'
inventory_src_file = r'/g/data/ha3/am7399/dev/passive-seismic/inventory/INVENTORY_20190116T144005.csv'
graphic_save_path = r'/g/data/ha3/am7399/temp'

In [7]:
cat = CatalogCSV(event_src_folder)

Reading /g/data/ha3/am7399/temp/CAT.csv


In [8]:
stations = [s for s in cat.station_dict.keys()]
print(stations[0:100])

['D30A', 'JSRW', 'KHLU', 'P57A', '736A', 'F19A', 'P53A', 'KEF4', 'LM203', 'CKN', 'SPCN', 'QCR', 'SFJWR', 'ASR', 'VGB', 'XAL', 'WI04', 'G21A', 'OUL', 'SGSI', 'TP2', 'MEZF', 'COPN', 'PAM', 'BW06', 'UPI', 'KCTX', 'BEY', 'MTTG', 'CBSW', 'W25A', 'CO03', 'MCK', 'SHI', 'JACH', 'KGE', 'PKD', 'K37A', 'SLBS', 'IMEL', 'KK07', 'L33A', 'AMANT', 'PUEL', 'DRBT', 'GUVIX', 'NC404', 'KBNM', 'HVS', 'HATZ', 'GDLN', 'U55A', 'X29A', 'GRFF', 'KWP', 'GHC', 'MAJO', 'PLOU', 'CDC', 'GRB4', 'OVMT', 'D46A', 'D47A', 'SJMB', 'CCSP', '453A', 'SABS', 'IRZ2', 'KBZ', 'JKUC', 'SUSD', 'C22A', 'JKE', 'ROSC', 'YAH', 'LRSO', 'VWUC', 'SIRN', 'EBR', 'GUAL3', 'PECO', 'MTW', 'BSSO', 'SIVA', 'IAS', 'JHC', 'MMA0', 'MK08', 'NGRK', 'PB06', 'RRRZ', 'SHOC', 'RKT', 'LCI', 'WR6', 'L20A', 'R38A', 'KRAI', 'ORX', 'TOVM']


In [9]:
# Load station inventory

In [10]:
inv = pd.read_csv(inventory_src_file, sep=',', parse_dates=['StationStart','StationEnd','ChannelStart','ChannelEnd'])

In [11]:
# Only using BHZ channels
inv = inv[inv['ChannelCode'] == 'BHZ']

In [12]:
inv_stations = list(inv['StationCode'].unique())
print("Event stations which are not known to the station inventory:\n{0}".format(
    list(set(stations) - set(inv_stations))))

Event stations which are not known to the station inventory:


['KAZ1',
 'QSDN',
 'SCTR',
 'CO06',
 'WUSB',
 'HAMB',
 'KEF4',
 'KAN08',
 'TV1H',
 'SCSP',
 'KRBS',
 'AUMHS',
 'MACA',
 'HALB',
 'LR03',
 'MESR',
 'DGTI',
 'CRQE',
 'C055',
 'ECBN',
 'MAR9',
 'TLBR',
 'G35D',
 'LM302',
 'J28D',
 'HLIG',
 'AUMAG',
 'KGE',
 'BO02',
 'BAHB',
 'JUMK',
 'A052A',
 'AUSSC',
 'J29M',
 'TAVE',
 'BHMN',
 'BORR',
 'GUVIX',
 'SVOC',
 'HWCB',
 'EUSB',
 'M27K',
 'TACV',
 'ETVZ',
 'ML02',
 'M26K',
 'LBB',
 'SHHT',
 'CO03',
 'GDU01',
 'ILOM',
 'FS10D',
 'LM118',
 'RACZ',
 'PQSS',
 'PORA',
 'PEIG',
 'G36D',
 'MUKA',
 'XTL',
 'SSHA',
 'TRNA',
 'BOSB',
 'WTLY',
 'KAN17',
 'DAMI2',
 'DACB',
 'FS08D',
 'VLDR',
 'BI04',
 'WADE',
 'FTIG',
 'IGOM',
 'INDI',
 'HATO',
 'HARO',
 'CNRM',
 'BOLE',
 'WWF',
 'BAKO',
 'LDUT',
 'CKRC',
 'GINR',
 'ONAU',
 'GLSP',
 'BUCA',
 'TRNY',
 'MT03',
 'NEWG',
 'GTIG',
 'LOBA2',
 'AUAYR',
 'TYRN',
 'ANBD',
 'OYDB',
 'G33D',
 'TSVK',
 'MALK',
 'NBCA',
 'NIND',
 'TAPR',
 'GOS1',
 'WUPA',
 'SLH',
 'CN2S',
 'JETT',
 'FAME',
 'GOND',
 'CRW',
 'CHIPN',


In [13]:
hour = np.timedelta64(1, 'h')

In [ ]:
#import pdb
agg_result = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
progress = tqdm(total=len(stations))
for stn in stations:
    progress.update()
    #pdb.set_trace()
    df_st = inv[inv['StationCode'] == stn]
    if df_st.empty:
        continue
    station_events = cat.station_dict[stn]['P']
    # Don't need to plot all events to detect distance error, a sampling of events will do.
    num_samples = min(100, len(station_events))
    for event_id, event_distance_deg in random.sample(station_events, num_samples):
        event = cat.event_dict[event_id]
        epicenter = event.preferred_origin.epicenter()
        event_date = np.datetime64(event.preferred_origin.utctime)
        station_mask = (df_st['StationStart'].isna() | ( \
              (df_st['StationStart'] < event_date) & 
              (df_st['StationEnd'].isna() | (df_st['StationEnd'] > (event_date + hour))))
            )
        channel_mask = df_st['ChannelStart'].isna() | \
            (
                (df_st['ChannelStart'] < event_date) & 
                (df_st['ChannelEnd'].isna() | (df_st['ChannelEnd'] > (event_date + hour)))
            )
        df_event = df_st[station_mask & channel_mask]
        if df_event.empty:
            continue
        computed_dist_deg = df_event.apply(
            lambda x: locations2degrees(x.Latitude, x.Longitude, epicenter[0], epicenter[1]), axis=1)
        for idx, dist in computed_dist_deg.iteritems():
            full_code = ".".join(df_event.loc[idx, ["NetworkCode", "StationCode"]].values)
            agg_result[stn][full_code]['Distance'].append(event_distance_deg)
            agg_result[stn][full_code]['ComputedDistance'].append(dist)
progress.close()

In [ ]:
def plotStationDistances(stn, stn_data, savepath=None):
    import itertools
    import os
    marker = itertools.cycle(('x', '1', '+', '2'))
    xval = [v['Distance'] for v in stn_data.values()]
    yval = [v['ComputedDistance'] for v in stn_data.values()]
    [plt.plot(x, y, marker=next(marker), markersize=10, linestyle='', fillstyle=None) for x, y in zip(xval, yval)]
    plt.axis('equal')
    plt.title(stn, fontsize=16)
    plt.legend(stn_data.keys(), fontsize=14)
    plt.grid()
    plt.gca().tick_params(axis='both', labelsize=14)
    plt.xlabel('Distance (deg)', fontsize=14)
    plt.ylabel('Computed Distance (deg)', fontsize=14)    
    if savepath:
        plt.savefig(os.path.join(savepath, stn + ".png"), dpi=150)
        plt.close()
    else:
        plt.show()

In [ ]:
# Dump images to files
[plotStationDistances(stn, stn_data, graphic_save_path) for stn, stn_data in agg_result.items()]
pass